In [1]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

import optuna
import lightgbm as lgb

import pickle

In [2]:
# !gsutil cp /home/clas_giulia_s/buckets/b1/datasets/competencia_02_fe_v01_undersampled.parquet /home/clas_giulia_s/datasets/

In [3]:
# base_path = '/home/clas_giulia_s/buckets/b1/'
base_path = '/Users/ignacio/MAESTRIA/DMEF/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
# dataset_file = 'competencia_02_fe_v01_undersampled.parquet'
dataset_file = 'competencia_01_fe_modelito_undersampled.csv'

ganancia_acierto = 273000
costo_estimulo = 7000

# data = pd.read_parquet(dataset_path + dataset_file)
data = pd.read_csv(dataset_path + dataset_file)

In [4]:
semillas = [5623, 292, 7494, 8504, 1663, 785, 5377, 4838, 2141, 2235, 9836, 1258, 3273, 8349, 1639, 1597, 3195, 40, 5186, 9278, 6281, 7515, 2046, 5642, 505, 4611, 3008, 2063, 2280, 1148, 618, 4806, 1503, 3926, 6363, 400, 2662, 9432, 1632, 386, 2545, 228, 1561, 3523, 4508, 9190, 8181, 7302, 6250, 7762, 8141, 6854, 622, 5327, 6379, 3867, 5420, 3030, 7275, 2040, 6042, 4365, 231, 8330, 8527, 2420, 2558, 9618, 3937, 555, 122, 4907, 7838, 5246, 100, 3243, 1449, 1052, 1906, 7657, 753, 4320, 4576, 9621, 8868, 8155, 7410, 2320, 6355, 1994, 7775, 8358, 3508, 3064, 3904, 3602, 5308, 6947, 1544, 624]

In [5]:

# meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
#                202001, 202002, 202003, 202004, 202005, 202006,
#                202007, 202008, 202009, 202010, 202011, 202012,
#                202101, 202102, 202103, 202104, 202105, 202106]  # entreno con todos los meses

meses_train = [202101, 202102, 202103, 202104]

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(69204, 679)

In [6]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [7]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [8]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] 
w_train = data['clase_peso']

In [9]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Entrenamiento

Cargamos el study de optuna que optimizamos en el script anterior

In [10]:

storage_name = "sqlite:///" + db_path + "optimizacion_lgbm_modelito_exp.db"
study_name = "competencia1_modelito_exp_lgbm" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-12-01 19:49:33,168] Using an existing study with name 'competencia1_modelito_exp_lgbm' instead of creating a new one.


In [11]:
resultados = study.trials_dataframe()
resultados.shape

(60, 12)

In [15]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

50

Reentrenamos los modelos individuales con la totalidad de los datos:

In [14]:
version = 'modelito_exp' # UPDATE

counter = 0

for semilla in semillas:

    
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': 0
    }

    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria,
                            weight=w_train)

    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    model.save_model(modelos_path + f'{version}/lgb_competencia1_{version}_s{semilla}_final.txt') # _final para indicar que es la version entrenada con todo el conjunto disponible.
    
    counter +=1
    print(f'{counter}: Modelo generado con semilla {semilla}: DONE')

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f